# STA 141B Data & Web Technologies for Data Analysis

### Lecture 3, 10/5/23, Memory handling


### Announcements

 - First HW due in one week! 

### Today's topics

 - Basics of Python (cont.')
 - Memory Handling in Python
     - Stack and Heap
     - Types
     - Reference Semantics
     - Interning

#### 3. Comprehensions and generators

A comprehension is a Python expression that transforms a sequence, element-by-element.

In [165]:
[x**2 for x in range(5)]

[0, 1, 4, 9, 16]

In [ ]:
x = [4, 3, 1]

In [2]:
x + 3

TypeError: can only concatenate list (not "int") to list

In [3]:
[y + 12 for y in x]

[16, 15, 13]

Think of this as Pythons `lapply`. You can include a condition in a comprehension:

In [172]:
y = [x**2 for x in range(11) if x % 2 == 0]
y 

[0, 4, 16, 36, 64, 100]

You can also iterate over subelements.

In [4]:
x = [[1, 2, 3], [4, 5, 6]] # print 1, 2, 3, 4, 5, 6

In [7]:
# somewhat clumsy
for sublist in x:
    for y in sublist:
        print(y)

1
2
3
4
5
6


In [8]:
[y for sublist in x for y in sublist]

[1, 2, 3, 4, 5, 6]

Be aware that `sublist in x` is the top loop and subloops are right thereof. In other words, the outermost iterables always come first in the comprehension.

A comprehension surrounded by `[ ]` is called a list comprehension and produces a <kbd>list</kbd>. A comprehension surrounded by `{ }` and including `:` is called a dictionary comprehension and produces a <kbd>dict</kbd>. Else it is called set comprehension. 

In [14]:
x = ["hello", "goodbye"]

lens = {len(name): (name,) for name in x} # print the length of names
lens

{5: ('hello',), 7: ('goodbye',)}

Remember that <kbd>dict</kbd> does not support equal keys and <kbd>set</kbd> does not support equal items, but <kbd>list</kbd> does. 

In [15]:
{x**2 for x in [-1, 0, 1]} # set # uniqueness of sets is checked with ==, not is

{0, 1}

There's no such thing as a tuple comprehension. Instead, a comprehension surrounded by `( )` is called a generator expression.

In [16]:
y = (x**2 for x in range(1001) if x % 2 == 0)
type(y)

generator

In [17]:
import sys
sys.getsizeof(y)

112

In [18]:
sys.getsizeof([x**2 for x in range(1001) if x % 2 == 0]) # produces a list, i.e., is evaluated

4216

Operating on a generator forces its evaluation. 

In [19]:
sum(y) 

167167000

This code does not produce any sensible result, because *a generator can only be used once*. Once iterated through, it is exhausted. Since this saves memory it is *much* more efficient than <kbd>list</kbd>.

In [20]:
for i in y:
    print(i, end=" ")

In [24]:
y = (x**2 for x in range(101) if x % 2 == 0)

In [25]:
for i in y:
    print(i, end=" ")

0 4 16 36 64 100 144 196 256 324 400 484 576 676 784 900 1024 1156 1296 1444 1600 1764 1936 2116 2304 2500 2704 2916 3136 3364 3600 3844 4096 4356 4624 4900 5184 5476 5776 6084 6400 6724 7056 7396 7744 8100 8464 8836 9216 9604 10000 

 The economics of memory show when we time operations. 

In [26]:
import timeit

In [27]:
print(timeit.timeit('''list_com = [i for i in range(100) if i % 2 == 0]''', number=1000000))
print(timeit.timeit('''gen_exp = (i for i in range(100) if i % 2 == 0)''', number=1000000))

4.750762458000054
0.28591766700014887


A generator is a special kind of iterable which computes its elements on demand. Examples are ranges and generator expressions. 
Generators are especially useful for working with data that are __too large__ to fit in memory. While making a huge list (say $10^9$ elements) might use enough memory to crash Python, making a generator with the same number of elements uses almost no memory. See more examples [here](https://zacks.one/python-generators/). 

Python's `itertools` module has functions for manipulating generators and iterable objects

### Stack and Heap

In [28]:
x = True
type(x)

bool

`x` is a variable, which corresponds to an <kbd>bool</kbd> object with value `True`. The variable itself holds merely a reference to a specific object. This reference is stored in local memory (the *stack*). Our compiler takes care in allocating stack memory, we don't have to do that. 

The <kbd>bool</kbd>-object and its value are stored on the random access memory (RAM, the *heap*). We can access the address of the object on the heap (and, conversely, the refrence on the stack): 

In [30]:
hex(id(x))

'0x100903530'

In [31]:
y = float(x)
hex(id(y))

'0x7fa4389ed770'

In Python, we can change the type of a variable.

In [32]:
hex(id(x))

'0x100903530'

In [33]:
x = int(x)
type(x)

int

In [34]:
hex(id(x))

'0x7fa43802e930'

<img src="../images/memory1.png" alt="" width="1000"/>

As soon as the `x`-variable, which previously referenced to the <kbd>bool</kbd> object is out of scope (either by deletion or recasting), the object on the heap is ready to be overwritten by the garbage collector. 



Let's work through the phrases: *Everything in Python is an object*. Some basic default objects (*types*) we have already met are 

- Numeric: <kbd>int</kbd>, <kbd>floats</kbd>, <kbd>complex</kbd>
- Boolean: <kbd>bool</kbd>
- String: <kbd>str</kbd>
- Sequence: <kbd>list</kbd>, <kbd>tuple</kbd>, <kbd>range</kbd>
- Mapping: <kbd>dict</kbd>

The function `sys.getsizeof` ([docs](https://docs.python.org/3/library/sys.html?highlight=getsizeof#sys.getsizeof)) returns the size in bytes of the object the variable points to. 

In [35]:
import sys
sys.getsizeof(x)

28

In [36]:
sys.getsizeof(y)

24

A <kbd>float</kbd> is less expensive than an <kbd>integer</kbd>. This is because <kbd>integer</kbd> stores additional information about size together with the actual value. The larger the integer, the more memory required. 

In [37]:
sys.getsizeof(100 ** 10)

36

In [38]:
sys.getsizeof(100.0 ** 10)

24

However, <kbd>integer</kbd> can store larger values than <kbd>float</kbd>. 

In [39]:
x = 500 ** 500 
type(x)

int

In [40]:
x

3054936363499604682051979393213617699789402740572326663893613909281291626524720457701857235108015228256875152693590467155317853427804283969735133114200917889630724420533772852222035588819531883700816508667930179487913663389937052516364978922702120035245082091219087448202119601494637211093403079855076782836518362040933993739599827677011489868164062500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [41]:
sys.getsizeof(x)

624

In [42]:
float(x)

OverflowError: int too large to convert to float

The function `range(start, stop, step)` ([docs](https://docs.python.org/3/library/stdtypes.html#range)) creates a <kbd>range</kbd> type object. It starts at `start` and ends at `stop - 1`, but does not instantiate an object of that length. 

In [43]:
x = range(0, 500**500)
sys.getsizeof(x)

48

In [44]:
sys.getsizeof(500**500)

624

A <kbd>tuple</kbd> is an ordered collection of values. Think of coordinates. <kbd>tuple</kbd> is immutable, which means they can't be changed after they're created.

In [45]:
x = 1, 3.0, "horse" # parenthesis are optional, but should be used for clarity 
x

(1, 3.0, 'horse')

In [46]:
type(x)

tuple

In [47]:
sys.getsizeof(x)

64

A <kbd>tuple</kbd> is inmutable. We have learned that once created, it can't be changed!

In [48]:
x[2] = 'horsies' 

TypeError: 'tuple' object does not support item assignment

In [50]:
try: x[2] = 'horsies' 
except: 
    print('Tuples are inmutable!')

Tuples are inmutable!


This is a feature, not shortcoming of <kbd>tuple</kbd>. Since they cannot be changed nor appended, they are more  economical than <kbd>list</kbd>. <kbd>list</kbd> is the mutable counterpart of <kbd>tuple</kbd>. They are instantiated with square brackets. 

In [51]:
y = [1, 3.0, "horse"]
y

[1, 3.0, 'horse']

In [52]:
type(y)

list

In [53]:
sys.getsizeof(y)

120

Lists are mutable, and in particular appendable. Since these actions are allowed, <kbd>list</kbd> objects require  more memory. The return of `sys.getsizeof` does not coincide with the values in the list! Instead, `y` is a variable with a reference to a <kbd>list</kbd> object on the heap, *which itself is a collection of adresses*. This collection of adresses takes $120$ bytes. 

In [54]:
sys.getsizeof(y)

120

In [55]:
sum([sys.getsizeof(i) for i in y])

106

In [56]:
sys.getsizeof(1) + sys.getsizeof(3.0) + sys.getsizeof("horse")

106

In contrast to <kbd>tuples</kbd>, they are however mutable. 

In [57]:
y[2] = "horsies"
y

[1, 3.0, 'horsies']

### Reference Semantics

Lists use *reference semantics*, which means that if you assign a list to two different variables, there's still only one list in memory, and both variables refer to it. As a result, changing the list with one variable changes the list for the other variable.

In [58]:
x = y

In [59]:
hex(id(x))

'0x7fa4201bb9c0'

In [60]:
hex(id(y))

'0x7fa4201bb9c0'

In [61]:
x[0] = "my"
y

['my', 3.0, 'horsies']

A new, non-referenced object can be created by slicing. 

In [62]:
z = y[:]

In [64]:
hex(id(z))

'0x7fa4201bb140'

In [73]:
z

['my', 3.0, 'horsies']

In [ ]:
z[1] = 3

In [77]:
hex(id(z[1]))

'0x7fa43802e970'

In [78]:
hex(id(y[1]))

'0x7fa4201b8070'

<img src="../images/memory2.png" alt="" width="1000"/>

Alternatively, we can use the copy method ([docs](https://docs.python.org/3/library/copy.html)) to the original list. 

In [80]:
z = y.copy()
hex(id(z))

'0x7fa4201bbd80'

In [81]:
hex(id(y))

'0x7fa4201bb9c0'

While the copies `y` and `z` are *equal*, the are not *identical*, because they point to different objects. 

In [82]:
y == z # equal

True

In [83]:
y is z # identical

False

In [84]:
y is x # identical

True

In [85]:
y[1] = 2
print(y)
print(z) 

['my', 2, 'horsies']
['my', 3.0, 'horsies']


Attention! This is a *shallow copy*, i.e., objects whithin the list will not be be reinstantiated! Above, the command `y[1] = 2` just instantiates a new <kbd>int</kbd> object of value `2` on the heap and replaces the former reference in `y` with the reference to that new object. 

In [86]:
hex(id(z[1])) == hex(id(y[1]))

False

This becomes tricky if the list references to another list: 

In [87]:
a = ['a', 'list']

In [88]:
y = [1, 2, 'three', a]

In [89]:
z = y.copy()

In [92]:
hex(id(y))

'0x7fa4201c8ec0'

In [93]:
hex(id(z))

'0x7fa4201caa80'

In [96]:
hex(id(y[3]))

'0x7fa4201ca600'

In [97]:
hex(id(z[3]))

'0x7fa4201ca600'

In [102]:
z[0] = 3

In [103]:
y

[1, 2, 'three', ['a', 'list']]

In [104]:
z

[3, 2, 'three', ['a', 'list']]

In [105]:
y[3][1] = 'ha'

In [106]:
print(y)
print(z)

[1, 2, 'three', ['a', 'ha']]
[3, 2, 'three', ['a', 'ha']]


In [107]:
hex(id(z[3])) == hex(id(y[3])) 

True

Although both lists are real copies, they reference to the same other list `a`, which has not been copied. 

In [108]:
hex(id(z[3])) == hex(id(y[3]))

True

This behaviour is irrespecive of the variable `a`. We can remove it from the scope. Since the list object `a` has pointed to still is in scope, it will not be taken by the garbage collector. 

In [109]:
hex(id(a))

'0x7fa4201ca600'

In [110]:
del(a)

In [111]:
hex(id(z[3]))

'0x7fa4201ca600'

We can copy the upper-level lists as well by calling the `copy.deepcopy`. 

In [ ]:
from copy import deepcopy
z = deepcopy(y)

In [113]:
y

[1, 2, 'three', ['a', 'ha']]

In [120]:
hex(id(z[1]))

'0x7fa43802e950'

In [119]:
hex(id(y[1]))

'0x7fa43802e950'

While the copies `y` and `z` are *equal*, the are not *identical*, because they point to different objects. 

In [121]:
y == z # equal

True

In [122]:
y is z # identical

False

### Interning 

The heap memory is memory that can be accessed and reserved by the programmer. Usually, this is tedious and automatically done. To optimize this process, Python uses *interning* to allocate ressources. Since `x` is merely a pointer to the <kbd>int</kbd> type object with value `1`, any other variable can point to the same adress.  

In [123]:
x = 1

In [124]:
y = 1

In [125]:
hex(id(x)) == hex(id(y))

True

This does not mean that integers use reference semantics! 

In [126]:
y = 2
x

1

In [127]:
hex(id(x)) == hex(id(y))

False

Integer internalization is only done from `-5` to `255`. 

In [133]:
x = 4.0
y = 4.0
hex(id(x)) == hex(id(y))

False

Interning works for several simple types: 

In [134]:
x = "Hi"
y = "Hi"

In [135]:
hex(id(x)) == hex(id(y))

True

Interning can be forced using `sys.intern`. 

In [1]:
a = "This is quite a long string."
b = "This is quite a long string."
hex(id(a)) == hex(id(b))

False

In [2]:
a = "This is quite a long string."
b = a
hex(id(a)) == hex(id(b))

True

In [3]:
import sys
a = sys.intern("This is quite a long string.")
b = sys.intern("This is quite a long string.")
hex(id(a)) == hex(id(b))

True

In [4]:
c = "This is quite a long string."
hex(id(a)) == hex(id(c))

False

When using `sys.intern`, the we can internalize an object without it being pointed to on the heap. 

In [5]:
a = sys.intern("This is quite a long string.")
hex(id(a))

'0x7f9a70bbaf80'

In [6]:
del a
b = sys.intern("This is quite a long string.")
hex(id(b))

'0x7f9a70bbaf80'

For reoccuring data, interning allows to use the heap economically. 

### Summary 

- There is stack and heap memory
- All objects are stored on the heap
- Lists are versatile, but generally inefficient
- Optimize heap usage via interning